In [ ]:
pip install transformers 

In [ ]:
import pandas as pd
import numpy as np

# reddit crawler
import praw

#BERT sentiment analysis
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

r = praw.Reddit(user_agent='...',
                client_id='...',
                client_secret='...',
                check_for_async=False)

In [ ]:
#Search any UBC course 
course_code = "CPSC 121"
#Number of posts to be scanned, the larger the better, but time consuming aswell. Max limit at 999
QueryLimit = 10

In [ ]:
ubc_reddit = r.subreddit("UBC")
titles = list(())
urls = list(())
comments = list(())
for post in ubc_reddit.search(course_code, limit=QueryLimit):
    titles.append(post.title)
    urls.append(post.url)
    submission = r.submission(id=post.id)
    if (post.link_flair_text != "Humour"):
        for top_level_comment in submission.comments:
            comm = top_level_comment.body
            if (len(comm) > 10 and len(comm) < 150):
                comments.append(comm)
    

In [ ]:
#perform sentiment analysis using BERT
BSA_comments = list(())
SumOfBR = 0
for i in comments:
    tokens = tokenizer.encode(i, return_tensors='pt')
    result = model(tokens)
    rank = int(torch.argmax(result.logits))+1
    BSA_comments.append(rank)
    SumOfBR += rank



In [ ]:
#Convert to CSV file to view each specific comment and it's BERT SA rating
df = pd.DataFrame({"Comments": comments, "BERT Ranking": BSA_comments})
df.to_csv(course_code + ".csv")

In [ ]:
#Calulate the average BERT rating of the course based on comments
 
final_bavg = SumOfBR / len(BSA_comments)

print(final_bavg)
